In [4]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import codecs
import time
import re
import os
from IPython.display import display, HTML

In [27]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

In [28]:
# headers = {
#     'User-Agent': 'Your Name, example.com',
#     'From': 'email@example.com'
# }

In [94]:

_line2 = "\"453453sva-hqxayena api435 vixiwavqwwAnwena2 ??amunA jihremi kimuwa(warhi_kiM_vakwavyam) apara-hqxayena (vixiwavqwwAnwena)"

_line2
_line1

[-1]

'"453453sva-hqxayena api435 vixiwavqwwAnwena2 ??amunA jihremi kimuwa(warhi_kiM_vakwavyam) apara-hqxayena (vixiwavqwwAnwena)'

'sva-hqxayena api vixiwavqwwAnwena amunA jihremi kimuwa'

In [22]:
fnames = ["lines_withno _brackets.txt","lines_brackets_converted.txt"]
count = 0     
for fname in fnames :
    problem = []
    problem.append(-1)
    fr = open("workdata/lines_submit/"+fname,encoding ='utf-8',mode='r')
    _lines = fr.readlines()
    fr.close()
    for _line in _lines :
#         if(count>20) :
#             break
        problem = []
        problem.append(-1)
        if(_line.startswith("filename : ")) :
            line1 = _line
            continue
        if(_line.startswith("lineno : ")) :
            line2 = _line
            continue
        if(_line.startswith("line : ")) :
            line3 = _line[7:]
            count += 1
            time.sleep(2)
            _line1 = line3[:-1]
            _line2 = re.sub(r'\d+|\"|\?|\.|\,|\*|\!|_\?|\-|_\ |\'',"",_line1)
            inputline = _line2
            if(inputline == " ") :
                print("emptyline")
                count = count-1
                continue
            s_d = inputline
            print(count)
            print(_line1)
            print(inputline)
#             fw = codecs.open("workdata/linetabledata/"+ str(count)+".txt",encoding = 'utf-8',mode = 'w')
#             fw.writelines("given line : " +_line1 + "\n")                                                        #file input
            s_c = s_d.replace(" ","+")
            urlname = ("http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=" +
                       s_c + "&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=")

            print(urlname)
            page = requests.get(urlname)
            soup = bs(page.text, 'html.parser')
        #     soup.prettify()
            table = soup.table
            tablebody = table.find('table',{'class' : 'center'})
            t = pd.DataFrame(columns = ['id','level','color_class','position','chunk_no','word','lemma','pre_verb','morph','colspan','wordlenth','aux_inf'])

            i = 0
            id_= 0
            for child in tablebody.children:
                if(child.name == 'tr') :
                    if i< 1 : 
                        linechar = []
                        c=0
                        for char in child.children :
                            linechar.append(char.string)
                            c += 1
                        i+=1
                        line = "".join(linechar)
                        linechunks = line.split("\xa0")
                        continue
                    position_ = 0
                    j =0
                    for wordtable in child.children:
            #             if(j < 20 ) :
            #                 j += 1
            #                 continue
                        c= 0
                        for ch in linechar[0:position_] :
                            if(re.match('\xa0',ch)) : 
                                c+=1
                        if(wordtable.contents):
                            color_ = wordtable.table.get('class')[0]
                            colspan_= wordtable.get('colspan')
                            word_ = wordtable.table.tr.td.string
                            onclickdatas_ = wordtable.table.tr.td.get('onclick')
                            for onclickdata_ in onclickdatas_.split("<br>") :
                                morphslist_ = re.findall(r'{ (.*?) }',onclickdata_)  #.split(' | ')
                                ldata = str(re.search(r'{.*?}\[(.*)\]',onclickdata_).group(1))
                                ldata = str(re.sub(r'</?a.*?>|</?i>',"",ldata))

                                lemmadata = ldata.split(" ")
                                if len(lemmadata) >1 :
                                    auxi_ = " ".join(lemmadata[1:])
                                else :
                                    auxi_ = ""
                                lemmas_ = "".join(lemmadata[0])
                                lemmalists_ = lemmas_.split("-")
                                if(len(lemmalists_) > 1) :
                                    preverb_ = ",".join(lemmalists_[0:(len(lemmalists_)-1)])
                                    lemmalist_ = "".join(lemmalists_[-1:]).split("_")
                                else :
                                    preverb_ = ""
                                    lemmalist_ = "".join(lemmalists_[0]).split("_")
                                if(len(lemmalist_) > 1):
                                    auxi_ =auxi_ + " sence of lemma = " + "".join(lemmalist_[1:(len(lemmalist_))])
                                    lemma_ = "".join(lemmalist_[0])
                                else : 
                                    lemma_ = "".join(lemmalist_[0])             
                                morphs_ = str(morphslist_[0])               
                                for morph_ in morphs_.split(" | ") : 
                                    t.loc[id_] = [id_,i,color_,position_,c+1,word_,lemma_,preverb_,morph_,colspan_,len(word_),auxi_]
                                    if(re.match(r'grey_back',color_)) :
                                        problem.append(id_)
                                    id_ += 1    


                            position_ += int(colspan_)
                #                 break
                        else : 
                            position_ += 1

            #         break

                    i = i+1
            if(len(problem)>1) :
                fn = "workdata/lines_submit/wrong_lines/"+ str(count)+".txt"
            else :
                fn = "workdata/lines_submit/correct_lines/"+ str(count)+".txt"
            
            fw = open(fn,encoding = 'utf-8' ,mode ='w')
            fw.writelines(line1)
            fw.writelines(line2)
            fw.writelines("given line : " +_line1 + "\n")
            fw.writelines("input line : " + inputline + "\n")
            fw.writelines("converted line : "+line + "\n")
            fw.writelines("converted line char :" + ",".join(linechar) + "\n")
            if(len(problem)>1) :
                _prob = []
                for prob in problem[1:]:
                    _prob.append(str(prob))
                fw.writelines("problem_yes ,index = ["+"-".join(_prob) + "]\n")
            else :
                fw.writelines("problem_no" + "\n")
            fw.close()
            t.to_csv(fn ,encoding = 'utf-8' , sep=',',index = False,mode = 'a' )
    #         display(t)
    #          count += 1 
    #         if(count > 2) :
    #             break
    #     break

# fr = open("data/sansk.txt",'r')
# print(fr.readlines())
# fr.close()

1
rAmasya BAryA sIwA wasya anujaH lakRmaNaH ca rAmeNa saha vanam gacCawaH
rAmasya BAryA sIwA wasya anujaH lakRmaNaH ca rAmeNa saha vanam gacCawaH
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=rAmasya+BAryA+sIwA+wasya+anujaH+lakRmaNaH+ca+rAmeNa+saha+vanam+gacCawaH&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
2
wApasAnAm bAlakAH ca wawra waranwi
wApasAnAm bAlakAH ca wawra waranwi
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=wApasAnAm+bAlakAH+ca+wawra+waranwi&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3
vqkReRu lawAH ca sanwi
vqkReRu lawAH ca sanwi
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=vqkReRu+lawAH+ca+sanwi&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
4
vqkRANAm lawAnAm ca SAKAsu vihagAH kUjanwi
vqkRANAm lawAnAm ca SAKAsu vihagAH kUjanwi
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=vqkRANAm+lawAnAm+ca+SAKAsu+vihagAH+kUjanwi&t=WX&topic=&mode=g&corpmode=&corpdir=&sentn

30
yaWAkAlam saH wayoH samIpam agacCaw ekam vqRaBam ca avaxaw
yaWAkAlam saH wayoH samIpam agacCaw ekam vqRaBam ca avaxaw
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=yaWAkAlam+saH+wayoH+samIpam+agacCaw+ekam+vqRaBam+ca+avaxaw&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
31
wawaH ekaH vqRaBaH xviwIyena waWA prahqwaH yaWA wasya vakRasaH ruXiram niragacCaw pIdayA ca saH xUram aXAvaw
wawaH ekaH vqRaBaH xviwIyena waWA prahqwaH yaWA wasya vakRasaH ruXiram niragacCaw pIdayA ca saH xUram aXAvaw
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=wawaH+ekaH+vqRaBaH+xviwIyena+waWA+prahqwaH+yaWA+wasya+vakRasaH+ruXiram+niragacCaw+pIdayA+ca+saH+xUram+aXAvaw&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
32
koZpi xviwIyena saha na awiRTaw na ca avaxaw
koZpi xviwIyena saha na awiRTaw na ca avaxaw
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=koZpi+xviwIyena+saha+na+awiRTaw+na+ca+avaxaw&t=WX&topic=&mode=g&corpmode=&corpdir=&sen

56
muniH viSvAmiwraH xaSaraWam upAgacCaw sahAyawAyE ca wam anvanayaw
muniH viSvAmiwraH xaSaraWam upAgacCaw sahAyawAyE ca wam anvanayaw
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=muniH+viSvAmiwraH+xaSaraWam+upAgacCaw+sahAyawAyE+ca+wam+anvanayaw&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
57
bAlyakAle  eva rAmaH ca lakRmaNaH ca xaSaraWasya AjFayA muneH viSvAmiwrasya yajFam rAkRaseByaH rakRiwum muninA saha vanam AgacCawAm
bAlyakAle  eva rAmaH ca lakRmaNaH ca xaSaraWasya AjFayA muneH viSvAmiwrasya yajFam rAkRaseByaH rakRiwum muninA saha vanam AgacCawAm
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=bAlyakAle++eva+rAmaH+ca+lakRmaNaH+ca+xaSaraWasya+AjFayA+muneH+viSvAmiwrasya+yajFam+rAkRaseByaH+rakRiwum+muninA+saha+vanam+AgacCawAm&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
58
viSvAmiwrasya ASramAw rAmaH ca lakRmaNaH ca miWilAm AgacCawAm
viSvAmiwrasya ASramAw rAmaH ca lakRmaNaH ca miWilAm AgacCawAm
http://sanskrit.inria.fr/cgi-bin/SK

83
BArawarARtriyA janawA yAxqSIm viSvabanXuwAm vAFCawi wAm viSvabanXuwAm prAcArayaw asmAkam rARtraxevawA
BArawarARtriyA janawA yAxqSIm viSvabanXuwAm vAFCawi wAm viSvabanXuwAm prAcArayaw asmAkam rARtraxevawA
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=BArawarARtriyA+janawA+yAxqSIm+viSvabanXuwAm+vAFCawi+wAm+viSvabanXuwAm+prAcArayaw+asmAkam+rARtraxevawA&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
84
paNdiwAnAm samAje apaNdiwAH mOnam BajeyuH
paNdiwAnAm samAje apaNdiwAH mOnam BajeyuH
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=paNdiwAnAm+samAje+apaNdiwAH+mOnam+BajeyuH&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
85
wawaH we sarve guFjAPalam pariwaH upaviSanwi
wawaH we sarve guFjAPalam pariwaH upaviSanwi
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=wawaH+we+sarve+guFjAPalam+pariwaH+upaviSanwi&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
86
kinwu we agnim na vinxanwi
kinwu we agnim na vinxanwi
http://san

121
SiSoH KaNdam mA kuru
SiSoH KaNdam mA kuru
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=SiSoH+KaNdam+mA+kuru&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
122
aham mama puwrasya jIvanam icCAmi
aham mama puwrasya jIvanam icCAmi
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=aham+mama+puwrasya+jIvanam+icCAmi&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
123
awaH SiSum xviwIyAyE nAryE yacCa
awaH SiSum xviwIyAyE nAryE yacCa
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=awaH+SiSum+xviwIyAyE+nAryE+yacCa&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
124
evam mama puwraH jIviRyawi
evam mama puwraH jIviRyawi
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=evam+mama+puwraH+jIviRyawi&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
125
xviwIyA asawyam vaxawi
xviwIyA asawyam vaxawi
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=xviwIyA+asawyam+vaxawi&t=WX&topic=&mode=

...

3468
  weRAM vivaSawA kA  aswi  
  weRAM vivaSawA kA  aswi  
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=++weRAM+vivaSawA+kA++aswi++&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3469
 wasya vivaSawA kA  aswi  
 wasya vivaSawA kA  aswi  
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=+wasya+vivaSawA+kA++aswi++&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3470
 puwraH mAwA piwA na ca  sanwi  
 puwraH mAwA piwA na ca  sanwi  
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=+puwraH+mAwA+piwA+na+ca++sanwi++&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3471
 kasya ixaM samAXisWalam  aswi  
 kasya ixaM samAXisWalam  aswi  
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=+kasya+ixaM+samAXisWalam++aswi++&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3472
 anena kIxqSaH we sambanXaH  aswi  
 anena kIxqSaH we sambanXaH  aswi  
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&c

3504
 BavAxqSAH anye mama cinwanaparAH BakwAH api mahawwamAH eva  sanwi  
 BavAxqSAH anye mama cinwanaparAH BakwAH api mahawwamAH eva  sanwi  
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=+BavAxqSAH+anye+mama+cinwanaparAH+BakwAH+api+mahawwamAH+eva++sanwi++&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3505
 nEyAyikaH wu aBAvaH cawurviXaH  Bavawi  prAgaBAvaH praXvamsABAvaH awyanwABAvaH anyonyABAvaH ca iwi nirxiSya rAjAnam apqcCaw 
 nEyAyikaH wu aBAvaH cawurviXaH  Bavawi  prAgaBAvaH praXvamsABAvaH awyanwABAvaH anyonyABAvaH ca iwi nirxiSya rAjAnam apqcCaw 
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=+nEyAyikaH+wu+aBAvaH+cawurviXaH++Bavawi++prAgaBAvaH+praXvamsABAvaH+awyanwABAvaH+anyonyABAvaH+ca+iwi+nirxiSya+rAjAnam+apqcCaw+&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3506
 sA palAyiwA, apahqwA, balAwkqwA, mAriwA vA?  aswi  
 sA palAyiwA apahqwA balAwkqwA mAriwA vA  aswi  
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us

3534
 mAwqBUmiH priyA  aswi  yasya na akAryam kuruwe hi saH
 mAwqBUmiH priyA  aswi  yasya na akAryam kuruwe hi saH
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=+mAwqBUmiH+priyA++aswi++yasya+na+akAryam+kuruwe+hi+saH&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3535
 ayam svaxeSaH eva we SikRiwasya api vikAryawAyAH hewuH  Bavawi  
 ayam svaxeSaH eva we SikRiwasya api vikAryawAyAH hewuH  Bavawi  
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=+ayam+svaxeSaH+eva+we+SikRiwasya+api+vikAryawAyAH+hewuH++Bavawi++&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3536
15 sarvam mama xuRkqwasya eva ixam Palam  aswi  
 sarvam mama xuRkqwasya eva ixam Palam  aswi  
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=+sarvam+mama+xuRkqwasya+eva+ixam+Palam++aswi++&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3537
 iha asya kaScana na xoRaH  aswi  
 iha asya kaScana na xoRaH  aswi  
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?l

3564
 yaxi me xaNdaxAnasya aXikAraH  aswi  warhi kRamAxAnasya api aXikAraH  aswi  
 yaxi me xaNdaxAnasya aXikAraH  aswi  warhi kRamAxAnasya api aXikAraH  aswi  
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=+yaxi+me+xaNdaxAnasya+aXikAraH++aswi++warhi+kRamAxAnasya+api+aXikAraH++aswi++&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3565
 ixaM yukwaM na  aswi  
 ixaM yukwaM na  aswi  
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=+ixaM+yukwaM+na++aswi++&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3566
 xaNdanIyaH eva  BavawA  
 xaNdanIyaH eva  BavawA  
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=+xaNdanIyaH+eva++BavawA++&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3567
 AScaryam mahaxAScaryaM  aswi  
 AScaryam mahaxAScaryaM  aswi  
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=+AScaryam+mahaxAScaryaM++aswi++&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3568
 yaxi evaM  aswi  w

3592
kaH XarmajFaH kqwajFaH sawyavAkyaH xqDavrawaH ca  aswi ?
kaH XarmajFaH kqwajFaH sawyavAkyaH xqDavrawaH ca  aswi 
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=kaH+XarmajFaH+kqwajFaH+sawyavAkyaH+xqDavrawaH+ca++aswi+&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3593
kaH ca vIryavAn  aswi ? 
kaH ca vIryavAn  aswi  
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=kaH+ca+vIryavAn++aswi++&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3594
kaH ca cAriwreNa yukwaH  aswi ?
kaH ca cAriwreNa yukwaH  aswi 
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=kaH+ca+cAriwreNa+yukwaH++aswi+&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3595
kaH sarvaBUweRu hiwaH  aswi ?
kaH sarvaBUweRu hiwaH  aswi 
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=kaH+sarvaBUweRu+hiwaH++aswi+&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3596
kaH vixvAn  aswi ?
kaH vixvAn  aswi 
http://sanskrit.inria.fr/cgi-bin/SKT/s

In [26]:
s = pd.read_csv("workdata/lines_submit/wrong_lines/23.txt",encoding = 'utf-8' ,sep = ',',skiprows = 7)
s

,id,level,color_class,position,chunk_no,word,lemma,pre_verb,morph,colspan,wordlenth,aux_inf
0,0,1,deep_sky_back,0,1,śrīḥ,śrī,NaN,acc. pl. f.,4,4,NaN
1,1,1,deep_sky_back,0,1,śrīḥ,śrī,NaN,nom. sg. f.,4,4,NaN
2,2,1,deep_sky_back,4,1,āmaḥ,āma,NaN,nom. sg. m.,4,4,NaN
3,3,1,yellow_back,9,2,śrī,śrī,NaN,iic.,3,3,NaN
4,4,1,deep_sky_back,12,2,kṛṣṇaḥ,kṛṣṇa,NaN,nom. sg. m.,6,6,NaN
5,5,1,deep_sky_back,19,3,satyavādī,satyavādin,NaN,nom. sg. m.,9,9,NaN
6,6,1,deep_sky_back,29,4,hariścandraḥ,hariścandra,NaN,nom. sg. m.,12,12,NaN
7,7,1,grey_back,42,5,satyapratijñabhīṣmaḥ,satyapratijñabhīṣmaḥ,NaN,?,19,20,NaN
8,8,1,yellow_back,62,6,mahā,mahat,NaN,iic.,4,4,NaN
9,9,1,deep_sky_back,70,6,buddhaḥ,buddha,NaN,nom. sg. m.,6,7,{ pp. }[budh_1]


In [ ]:
_line2 = re.sub(r'_ ',"","hggj_h_ (gf")
_line2


In [30]:
#     s_d = inputline
#     s_c = s_d.replace(" ","+")
#     urlname = ("http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=" +
#                s_c + "&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=")

#     print(urlname)
#     time.sleep(2)
#     page = requests.get(urlname)

In [31]:
#     soup = bs(page.text, 'html.parser')
#     soup.prettify()

In [32]:
# table = soup.table
# print(table)

In [33]:
# tablebody = table.find('table',{'class' : 'center'})
# print(tablebody)

In [34]:
# t = pd.DataFrame(columns = ['id','level','color_class','position','chunk_no','word','lemma','pre_verb','morph','colspan','wordlenth','aux_inf'])
# t


In [35]:
# for child in tablebody.children:
#     if(child.name == 'tr') :
#         print(child.name)

In [36]:
# 'id','level','color_class','position','chunk_no','word','lemma','morph','colspan','wordlenth','pre_verb','aux_inf'

In [37]:
# i = 0
# id_= 0
# for child in tablebody.children:
#     if(child.name == 'tr') :
#         if i< 1 : 
#             linechar = []
#             c=0
#             for char in child.children :
#                 linechar.append(char.string)
#                 c += 1
#             i+=1
#             line = "".join(linechar)
#             linechunks = line.split("\xa0")
#             continue
#         position_ = 0
#         j =0
#         for wordtable in child.children:
# #             if(j < 20 ) :
# #                 j += 1
# #                 continue
#             c= 0
#             for ch in linechar[0:position_] :
#                 if(re.match('\xa0',ch)) : 
#                     c+=1
#             if(wordtable.contents):
#                 color_ = wordtable.table.get('class')[0]
#                 colspan_= wordtable.get('colspan')
#                 word_ = wordtable.table.tr.td.string
#                 onclickdatas_ = wordtable.table.tr.td.get('onclick')
#                 for onclickdata_ in onclickdatas_.split("<br>") :
#                     morphslist_ = re.findall(r'{ (.*?) }',onclickdata_)  #.split(' | ')
#                     ldata = str(re.search(r'{.*?}\[(.*)\]',onclickdata_).group(1))
#                     ldata = str(re.sub(r'</?a.*?>|</?i>',"",ldata))

#                     lemmadata = ldata.split(" ")
#                     if len(lemmadata) >1 :
#                         auxi_ = " ".join(lemmadata[1:])
#                     else :
#                         auxi_ = ""
#                     lemmas_ = "".join(lemmadata[0])
#                     lemmalists_ = lemmas_.split("-")
#                     if(len(lemmalists_) > 1) :
#                         preverb_ = ",".join(lemmalists_[0:(len(lemmalists_)-1)])
#                         lemmalist_ = "".join(lemmalists_[-1:]).split("_")
#                     else :
#                         preverb_ = ""
#                         lemmalist_ = "".join(lemmalists_[0]).split("_")
#                     if(len(lemmalist_) > 1):
#                         auxi_ =auxi_ + " sence of lemma = " + "".join(lemmalist_[1:(len(lemmalist_))])
#                         lemma_ = "".join(lemmalist_[0])
#                     else : 
#                         lemma_ = "".join(lemmalist_[0])             
#                     morphs_ = str(morphslist_[0])               
#                     for morph_ in morphs_.split(" | ") : 
#                         t.loc[id_] = [id_,i,color_,position_,c+1,word_,lemma_,preverb_,morph_,colspan_,len(word_),auxi_]
#                         id_ += 1    


#                 position_ += int(colspan_)
#     #                 break
#             else : 
#                 position_ += 1

# #         break
        
#         i = i+1

        
# # print(color_)
# # print(colspan_)
# # print(position_)
# # print(word_)
# # print(onclickdata_)
# # print(ldata)
# # print(morphslist_)
# # print(lemmalists_)
# # print(auxi_)
# # print(lemma_)
# # print(morphs_)
# # print(preverb_)
# # print(linechar) 
# print(line)
# # c= 0
# # for ch in linechar[0:position_] :
# #     if(re.match('\xa0',ch)) : 
# #         c+=1
# # print(c)
# t